<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Importations" data-toc-modified-id="Importations-1">Importations</a></span></li><li><span><a href="#Ajout-de-colonnes-depuis-'date'" data-toc-modified-id="Ajout-de-colonnes-depuis-'date'-2">Ajout de colonnes depuis 'date'</a></span></li><li><span><a href="#Ajout-des-coordonnées" data-toc-modified-id="Ajout-des-coordonnées-3">Ajout des coordonnées</a></span></li></ul></div>

In [53]:
import numpy.linalg as npl
import pandas as pd
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt

# Importations

In [41]:
train = pd.read_csv("Train/Train/X_station_train.csv")
coords = pd.read_csv("Other/Other/stations_coordinates.csv")

# Ajout de colonnes depuis 'date'

In [34]:
train["month"] = train["date"].replace(to_replace="^.{5}", value="", regex=True)\
                              .replace(to_replace=".{12}$", value="", regex=True)

train["day"] = train["date"].replace(to_replace="^.{8}", value="", regex=True)\
                            .replace(to_replace=".{9}$", value="", regex=True)

train["hour"] = train["date"].replace(to_replace="^.{11}", value="", regex=True)\
                             .replace(to_replace=".{6}$", value="", regex=True)

In [36]:
del train["date"]

In [38]:
train = train.astype({"month": "int8", "day": "int8", "hour": "int8"})

# Ajout des coordonnées

In [42]:
train = train.merge(coords, how="left", on="number_sta")

In [51]:
train.groupby("number_sta", as_index=False).mean()

,number_sta,ff,t,td,hu,dd,precip,lat,lon,height_sta
0,14066001,4.255939,284.944430,281.595258,80.800802,196.742004,0.079711,49.334,-0.431,2.0
1,14126001,NaN,284.616017,282.325057,87.140469,NaN,0.062474,49.145,0.042,125.0
2,14137001,4.016765,284.568835,281.295591,81.792968,204.500866,0.076184,49.180,-0.456,67.0
3,14216001,3.034672,284.497981,280.931299,80.531124,207.596797,0.081724,48.928,-0.149,155.0
4,14296001,NaN,283.417831,NaN,NaN,NaN,0.128194,48.795,-1.037,336.0
...,...,...,...,...,...,...,...,...,...,...
262,86149001,NaN,281.006329,NaN,NaN,NaN,0.089744,46.917,0.025,60.0
263,86165005,2.691349,285.729780,281.051342,76.765650,179.069880,0.079490,46.412,0.841,153.0
264,86272002,NaN,285.615292,NaN,NaN,NaN,0.075736,46.839,0.457,120.0
265,91200002,2.029602,284.325491,280.864879,81.702263,185.769466,0.080942,48.526,1.993,116.0


In [61]:
train.sample(500).pivot(columns="number_sta")

date                                                        \
number_sta 14066001 14126001 14137001 14216001 14296001 14357002 14366002   
12582           NaN      NaN      NaN      NaN      NaN      NaN      NaN   
16056           NaN      NaN      NaN      NaN      NaN      NaN      NaN   
31137           NaN      NaN      NaN      NaN      NaN      NaN      NaN   
34486           NaN      NaN      NaN      NaN      NaN      NaN      NaN   
43084           NaN      NaN      NaN      NaN      NaN      NaN      NaN   
...             ...      ...      ...      ...      ...      ...      ...   
4380052         NaN      NaN      NaN      NaN      NaN      NaN      NaN   
4384599         NaN      NaN      NaN      NaN      NaN      NaN      NaN   
4401824         NaN      NaN      NaN      NaN      NaN      NaN      NaN   
4403789         NaN      NaN      NaN      NaN      NaN      NaN      NaN   
4405119         NaN      NaN      NaN      NaN      NaN      NaN      NaN   

                                       ... height_sta                    \
number_sta 14372001 14501002 14515001  ...   86021002 86027001 86039001   
12582           NaN      NaN      NaN  ...        NaN      NaN      NaN   
16056           NaN      NaN      NaN  ...        NaN      NaN      NaN   
31137           NaN      NaN      NaN  ...        NaN      NaN      NaN   
34486           NaN      NaN      NaN  ...        NaN      NaN      NaN   
43084           NaN      NaN      NaN  ...        NaN      NaN      NaN   
...             ...      ...      ...  ...        ...      ...      ...   
4380052         NaN      NaN      NaN  ...        NaN      NaN      NaN   
4384599         NaN      NaN      NaN  ...        NaN      NaN      NaN   
4401824         NaN      NaN      NaN  ...        NaN      NaN      NaN   
4403789         NaN      NaN      NaN  ...        NaN      NaN      NaN   
4405119         NaN      NaN      NaN  ...        NaN      NaN      NaN   

                                                                           
number_sta 86092002 86096001 86137003 86165005 86272002 91200002 95690001  
12582           NaN      NaN      NaN      NaN      NaN      NaN      NaN  
16056           NaN      NaN      NaN      NaN      NaN      NaN      NaN  
31137           NaN      NaN      NaN      NaN      NaN      NaN      NaN  
34486           NaN      NaN      NaN      NaN      NaN      NaN      NaN  
43084           NaN      NaN      NaN      NaN      NaN      NaN      NaN  
...             ...      ...      ...      ...      ...      ...      ...  
4380052         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
4384599         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
4401824         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
4403789         NaN      NaN      NaN      NaN      NaN      NaN      NaN  
4405119         NaN      NaN      NaN      NaN      NaN      NaN      NaN  

[500 rows x 2486 columns]

In [55]:
npl.norm(train["lat"][0] - train["lon"][0])

49.765

In [57]:
train

,number_sta,date,ff,t,td,hu,dd,precip,Id,lat,lon,height_sta
0,14066001,2016-01-01 00:00:00,3.05,279.28,277.97,91.4,200.0,0.0,14066001_0_0,49.334,-0.431,2.0
1,14066001,2016-01-01 01:00:00,2.57,278.76,277.45,91.4,190.0,0.0,14066001_0_1,49.334,-0.431,2.0
2,14066001,2016-01-01 02:00:00,2.26,278.27,277.02,91.7,181.0,0.0,14066001_0_2,49.334,-0.431,2.0
3,14066001,2016-01-01 03:00:00,2.62,277.98,276.95,93.0,159.0,0.0,14066001_0_3,49.334,-0.431,2.0
4,14066001,2016-01-01 04:00:00,2.99,277.32,276.72,95.9,171.0,0.0,14066001_0_4,49.334,-0.431,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4409469,95690001,2017-12-30 19:00:00,9.10,286.68,283.44,80.8,239.0,0.0,95690001_729_19,49.108,1.831,126.0
4409470,95690001,2017-12-30 20:00:00,8.58,286.39,283.21,81.1,231.0,0.0,95690001_729_20,49.108,1.831,126.0
4409471,95690001,2017-12-30 21:00:00,8.74,286.28,283.40,82.6,226.0,0.0,95690001_729_21,49.108,1.831,126.0
4409472,95690001,2017-12-30 22:00:00,9.04,286.21,283.29,82.4,224.0,0.0,95690001_729_22,49.108,1.831,126.0
